In [12]:
import getpass
import os
import time
from dotenv import load_dotenv

# Load the .env file
_ = load_dotenv()

### Model

In [13]:
from agno.models.ollama import Ollama

sql_ollama_model=Ollama(id="qwen2.5-coder:7b")

### Text 2 sql agent

In [32]:
from agno.agent import Agent

instructions="""
You are an expert in SQL. You can create sql queries from natural language following these schemas:

CREATE TABLE "Orders" (
	date DATETIME, 
	id TEXT, 
	quantity FLOAT
)

CREATE TABLE "Stock" (
	id TEXT, 
	current_stock_quantity FLOAT, 
	units TEXT, 
	avg_lead_time_days BIGINT, 
	maximum_lead_time_days BIGINT, 
	unit_price FLOAT
)

Return only the sql query. Don't explain the query and don't put the sql query in ```sql\n```.
"""
text2sql_agent = Agent(model=sql_ollama_model, name="text2sql", instructions=instructions, show_tool_calls=True)

### Tests

In [33]:
def write_sql_query(query:str):
    """ Function to write sql query """
    start_time = time.perf_counter()

    response = text2sql_agent.run(query)

    end_time = time.perf_counter()
    
    latency = end_time - start_time
    
    print(f"Execution time: {latency} seconds")
    answer = response.content
    
    return answer
    

In [19]:
query = "How many orders are there in the Orders table?"
write_sql_query(query)

Execution time: 34.3474353000056 seconds


'SELECT COUNT(*) FROM Orders;'

In [34]:
query = "Select the orders that have a quantity equal 475 in the Orders table."
write_sql_query(query)

Execution time: 40.769207000004826 seconds


'SELECT * FROM Orders WHERE quantity = 475;'

In [35]:
query = "Get all records from the Stock table."
write_sql_query(query)

Execution time: 5.330851200007601 seconds


'SELECT * FROM Stock;'

In [38]:
query = "List the IDs of products in stock with their current quantity"
write_sql_query(query)

Execution time: 9.713036499975715 seconds


'SELECT id, current_stock_quantity FROM Stock WHERE current_stock_quantity > 0;'

In [39]:
query = "Find products with more than 10,000 units in stock"
write_sql_query(query)

Execution time: 9.001096500025596 seconds


'SELECT id FROM Stock WHERE current_stock_quantity > 10000;'

In [40]:
query = "Get the total quantity of products in stock"
write_sql_query(query)

Execution time: 7.69951229999424 seconds


'SELECT SUM(current_stock_quantity) as total_quantity FROM Stock;'

In [41]:
query = "Get the product with the highest maximum lead time"
write_sql_query(query)

Execution time: 6.523983900027815 seconds


'SELECT id FROM Stock ORDER BY maximum_lead_time_days DESC LIMIT 1'

In [42]:
query = "List products in stock that have been ordered at least once"
write_sql_query(query)

Execution time: 11.286583699984476 seconds


'SELECT DISTINCT s.id FROM Stock s JOIN Orders o ON s.id = o.id WHERE o.quantity > 0'

### Benchmark

In [43]:
import json
import glob

def load_benchmark(benchmark_path):
    """ This function loads the benchmark dataset"""
    questions = []
    expected_sql = []
    with open(benchmark_path, 'r', encoding='utf-8') as f:
        try:
            datos = json.load(f)
            questions.extend(datos.keys())
            expected_sql.extend(datos.values())
        except Exception as e:
            print(f,e)
    return questions, expected_sql

questions , expected_sql = load_benchmark("../tests/dataset_queries_en.json")

In [44]:
questions

['Get all records from the Stock table',
 'List the IDs of products in stock with their current quantity',
 'Get the average unit price of all products in stock',
 'Find products with more than 10,000 units in stock',
 'Get the total quantity of products in stock',
 'List orders placed in the year 2023',
 'Get the product with the highest maximum lead time',
 'List products whose unit price is above the average',
 'Get the total number of orders per product',
 'List products in stock that have been ordered at least once']

In [45]:
expected_sql

['SELECT * FROM Stock;',
 'SELECT id, current_stock_quantity FROM Stock;',
 'SELECT AVG(unit_price) AS avg_unit_price FROM Stock;',
 'SELECT id, current_stock_quantity FROM Stock WHERE current_stock_quantity > 10000;',
 'SELECT SUM(current_stock_quantity) AS total_stock FROM Stock;',
 "SELECT * FROM Orders WHERE strftime('%Y', date) = '2023';",
 'SELECT id, maximum_lead_time_days FROM Stock ORDER BY maximum_lead_time_days DESC LIMIT 1;',
 'SELECT id, unit_price FROM Stock WHERE unit_price > (SELECT AVG(unit_price) FROM Stock);',
 'SELECT id, COUNT(*) AS total_orders FROM Orders GROUP BY id;',
 'SELECT DISTINCT s.id, s.current_stock_quantity FROM Stock s JOIN Orders o ON s.id = o.id;']

In [46]:
import pandas as pd
import time

def eval_benchmark(benchmark_path:str)->pd.DataFrame:
	""" This function evaluates the benchmark dataset"""

	# Load the benchmark dataset
	questions , expected_sql = load_benchmark(benchmark_path)
	# Create a dataframe to store the results
	df = pd.DataFrame({"question":questions, "expected_sql":expected_sql})

	latencies = []
	answers = []
	for index, row in df.iterrows():
		try:
			question = row["question"]
			expected_sql = row["expected_sql"]
			
			# Measure the latency
			start_time = time.time()
			# Invoke the agent
			response = text2sql_agent.run(query)
			predicted_sql = response.content
			# Measure the latency
			latency = time.time() - start_time
			latencies.append(latency)
			# Store the predicted SQL query
			answers.append(predicted_sql)
		except Exception as e:
			answers.append("ERROR")
			latencies.append(-1)
			print(f"Error processing question: {question}")

	# Store the results in the dataframe
	df["predicted_sql"] = answers
	df["latency"] = latencies
	return df

In [47]:
df_eval = eval_benchmark( "../tests/dataset_queries_en.json")

In [49]:
df_eval.head(10)

,question,expected_sql,predicted_sql,latency
0,Get all records from the Stock table,SELECT * FROM Stock;,SELECT DISTINCT s.id FROM Stock s JOIN Orders ...,7.878024
1,List the IDs of products in stock with their c...,"SELECT id, current_stock_quantity FROM Stock;",SELECT DISTINCT id FROM Stock WHERE id IN (SEL...,5.383047
2,Get the average unit price of all products in ...,SELECT AVG(unit_price) AS avg_unit_price FROM ...,SELECT DISTINCT id FROM Orders INTERSECT SELEC...,4.166067
3,"Find products with more than 10,000 units in s...","SELECT id, current_stock_quantity FROM Stock W...","SELECT s.id, s.current_stock_quantity, s.units...",18.221512
4,Get the total quantity of products in stock,SELECT SUM(current_stock_quantity) AS total_st...,SELECT DISTINCT id FROM Orders WHERE id IN (SE...,6.962802
5,List orders placed in the year 2023,"SELECT * FROM Orders WHERE strftime('%Y', date...",SELECT DISTINCT Orders.id FROM Orders JOIN Sto...,8.252865
6,Get the product with the highest maximum lead ...,"SELECT id, maximum_lead_time_days FROM Stock O...","SELECT s.id, s.current_stock_quantity, s.units...",16.158268
7,List products whose unit price is above the av...,"SELECT id, unit_price FROM Stock WHERE unit_pr...",SELECT id FROM Orders GROUP BY id HAVING COUNT...,7.139036
8,Get the total number of orders per product,"SELECT id, COUNT(*) AS total_orders FROM Order...",SELECT id FROM Orders WHERE id IN (SELECT id F...,6.569498
9,List products in stock that have been ordered ...,"SELECT DISTINCT s.id, s.current_stock_quantity...",SELECT DISTINCT Stock.id FROM Stock JOIN Order...,7.271968


In [50]:
model_id = "qwen2.5-coder"

df_eval.to_csv(f"../tests/df_benchmark_results_{model_id}.csv", index=False)

In [1]:
import pandas as pd

model_id = "qwen2.5-coder"

df_eval = pd.read_csv(f"../tests/df_benchmark_results_{model_id}.csv")

In [2]:
import plotly.graph_objects as go

# Crear figura
fig = go.Figure(data=[go.Bar(x=df_eval.question, y=df_eval.latency)])

# Configurar ejes
fig.update_layout(
    title=f'Our Text2SQL Benchmark dataset to evaluate the performance of : {model_id}',
    xaxis_title='Question',
    yaxis_title='Latency (s)'
)
fig.update_layout(title=dict(font_size=12, font_weight='bold', x=0.5))
fig.update_traces(marker_line_color='#e69138', marker_line_width=2, marker_color="#f35a02")
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', yaxis=dict(gridcolor='lightgrey'), height=600, width=1000)

# Mostrar figura
fig.show()